# This code generates the usage plots for the Symbolic reasoning model and LLMs

In [ ]:

# Reproducibility & plotting config
import os, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

np.random.seed(42)

FIG_DIR = Path("figures")
FIG_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "figure.figsize": (6, 4),
    "font.size": 10,
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "axes.linewidth": 0.8,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
})

def savefig(name: str, tight=True):
    """Save figure to both PNG and PDF in FIG_DIR."""
    path_png = FIG_DIR / f"{name}.png"
    path_pdf = FIG_DIR / f"{name}.pdf"
    if tight:
        plt.tight_layout()
    plt.savefig(path_png, bbox_inches="tight")
    plt.savefig(path_pdf, bbox_inches="tight")
    print(f"Saved: {path_png} and {path_pdf}")


# Rules based model 

#### Necessary package imports

In [ ]:
import pandas as pd
import numpy as np
import wordfreq
import json
import os
import re
import time
import sys
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import psutil

os.getcwd()

In [ ]:
process = psutil.Process(os.getpid())

# Measure memory before loading the model
mem_before = process.memory_info().rss

#### Train/Test Splits

In [ ]:
df_train = pd.read_csv('gdrive/My Drive/Colab Notebooks/clinical_training.csv', index_col=0)
df_test = pd.read_csv('gdrive/My Drive/Colab Notebooks/clinical_testing.csv', index_col=0)

In [ ]:
df_train_virtassist = df_train[df_train['dataset'] == 'virtassist']
df_test_virtassist = df_test[df_test['dataset'] == 'virtassist']

In [ ]:
df_train_virtscribe = df_train[df_train['dataset'] == 'virtscribe']
df_test_virtscribe = df_test[df_test['dataset'] == 'virtscribe']

In [ ]:
df_train_aci = df_train[df_train['dataset'] == 'aci']
df_test_aci = df_test[df_test['dataset'] == 'aci']

#### Global method definitions

In [ ]:
# col : name of column in dataframe that you want to interrogate for keywords
# indicators : list of keyword arguments
# df : dataframe of interest

# kwarg_counter : returns (A) t_list : a list of row indicies from the dataframe that do not contain any keywords from indicators
#                         (B) t_dict : a list of the frequency of occurence of each word in indicators within the dataframe's column
#                         (C) cc : the reading frame of the doctor's passage of speech that contains a keyword specified in "indicators"
def kwarg_counter(col, indicators, df=df_train):

    # t_ind : this index
    # t_list : this list of indices
    # t_dict : dictionary containing the indicators and their frequency in df[col]
    t_ind = 0
    t_list = []
    t_dict = dict()

    # initialize the dict
    for i in indicators:
        t_dict[i] = 0
    t_dict['other'] = 0

    # cc : contains chief complaint reading frames
    cc = dict()

    # for each row in df[col]:
    #      iterate through the list of indicators
    #           if one indicator in the list of indicators is present among the first 400 characters of the dialogue,
    #                 increment t_dict[indicator] and add the whole passage of text to cc (reading frame).
    for instance in df[col].values:
        t = 0
        cc_members = []
        for ind in indicators:
            if ind in instance[0:400]:
                cc_members.append(instance[instance.index(ind) + len(ind):].split('\n')[0][0:-1])
                t_dict[ind] += 1
                t+=1
        if t == 0:
            t_list.append(t_ind)
            t_dict['other'] += 1
        cc[t_ind] = cc_members
        t_ind+=1

    return t_list, t_dict, cc

In [ ]:
# count_between_ranges : UNUSED method now, but originally intended to observe the frequency of keywords between different ranges
def count_between_ranges(list_ref, list_pos):

    # sort list_pos to avoid mismatched ranges
    list_pos.sort()

    # initialize a list to store the counts for each range
    counts = []

    # loop through each pair of consecutive values in list_pos
    for i in range(len(list_pos) - 1):
        start = list_pos[i]
        end = list_pos[i + 1]

        # count the number of elements in list_ref that fall within the range [start, end]
        count = sum(start < x < end for x in list_ref)
        counts.append(count)

    return counts

In [ ]:
# Given: A string containing a patient-physician conversation, stored in 'dialogue' column of dataframe
# returns: A string containing exclusively the doctor's speech, denoted by [doctor] in the text
def doctor_from_dialogue(dialogue):

    exchanges = dialogue.split('\n')
    doc_sentences = []

    for e in exchanges:
        if '[doctor]' in e:
            doc_sentences.append(e)

    return doc_sentences

lo_convo = []
for convo in df_train['dialogue'].values:
    ds_convo = doctor_from_dialogue(convo)
    lo_convo.append('\n'.join(ds_convo))

df_train['doc_dialogues'] = lo_convo

#### Exploratory analysis

##### Are the outputs notes from genAI methods standardized? Mostly, yes. 88% of notes outputs in the training set include the words "CHIEF COMPLAINT" and 5.97% exclude a chief complaints section entirely.

In [ ]:
idx_of_missing_CC, dict_of_counts, cc_reading_frames_from_notes = kwarg_counter('note', ['CHIEF COMPLAINT\n\n', 'CC:\n\n'])
dict_of_counts

In [ ]:
idx_of_missing_CC

#### Are there words used consistently in dialogue to indicate a chief complaint? Partially.

In [ ]:
t_list_dialogue_1, t_dict_dialogue_1, cc_from_dialogue_1 = kwarg_counter('dialogue', ['present', 'eval'])
t_dict_dialogue_1

##### What other words can be included?

In [ ]:
t_list_dialogue_2, t_dict_dialogue_2, cc_from_dialogue_2 = kwarg_counter('dialogue', ['present', 'eval', 'here for', 'here with', 'for'])
t_dict_dialogue_2

##### There is full coverage in the dataset using these five keywords! Essential to make sure that "for" is within a reading frame of a chief complaint (which is an analysis we have conducted separately)

### Let's look at just the virtassist subset.

In [ ]:
df_train_virtassist.head()
#df_test_virtassist.head()

#### Are there keywords in virtassist that indicate chief complaint? Looks like it!

In [ ]:
virtassist_inds, virtassist_dict, virtassist_frames = kwarg_counter('dialogue', ['present', 'eval', 'here for', 'here with'], df_train_virtassist)

In [ ]:
virtassist_dict

#### Let's clean the reading frames from the keywords identified above, and turn each reading frame into a list of tokens.

In [ ]:
# reading_frames : a list of reading frames

# reading_frame_cleaner : splits the reading frame to only contain the first sentence of the dialogue.
# Returns a list of cleaned reading frames
def reading_frame_cleaner(reading_frames):

    # list of "very accurate" chief complaints (it actually stood for virtassist; too lazy to change now)
    va_cc = []

    # for a frame in reading frames,
    #     if there is something in the reading frame,
    #          append everything between the first english character and the period to va_cc
    #     otherwise, append NULL
    for t in reading_frames:
        if reading_frames[t] != []:
            first_part = reading_frames[t][0].split('.')[0]
            if " " in first_part:
                va_cc.append(first_part.split(" ", 1)[1])
            else:
                va_cc.append('NULL')
        else:
            va_cc.append('NULL')

    return va_cc

#va_cc = reading_frame_cleaner(virtassist_frames)
#va_cc[0:10]

##### Now I can tokenize words in a reading frame to compute the freqencies of each token in the english language using the wordfreq library: calculating the zipf (log-10 occurrences per million words) for each token

In [ ]:
# lo_frames : list of cleaned reading frames

# frame_tokenizer : splits a reading frame into individual words
def frame_tokenizer(lo_frames):

    lo_tkn = []

    # for each reading frame, split the frame by words (space character) and append to lo_tkn
    for phrase in lo_frames:
        lo_tkn.append(phrase.split(' ')[0:-1])

    return lo_tkn

#lo_tkn = frame_tokenizer(va_cc)
#lo_tkn[0:4]

In [ ]:
# lo_tkn : a list containing a list of tokens (words)

# freq_calc : calculate the zipf's law word frequency using a dictionary of 1 million english words.
def freq_calc(lo_tkn):

    lo_freqs = []

    # for a tokenized reading frame,
    #     for each token,
    #          fetch the zipf frequency and append to series_freqs
    # append all the series freqs (corresponding to reading frames) together in lo_freqs
    for tkn_series in lo_tkn:
        series_freqs = []
        for tkn in tkn_series:
            series_freqs.append(wordfreq.zipf_frequency(tkn, 'en', wordlist='small'))
        lo_freqs.append(series_freqs)
    return lo_freqs

#lo_freqs = freq_calc(lo_tkn)
#lo_freqs[0:4]

#### Can we identify medical breaks in the list of tokens based on a significant change in frequency score? Let's try with a delta of 1.0

In [ ]:
# lo_tkn and lo_freqs are previously defined

# frequency_parser : identifies a breakpoint where the change in
def frequency_parser(lo_tkn, lo_freqs):

    freq_subsetter_idx = 0
    cc_fisher = []
    while freq_subsetter_idx < len(lo_tkn):

        word_list = lo_tkn[freq_subsetter_idx]
        frequency_list = lo_freqs[freq_subsetter_idx]

        # Initialize variable to store the index after the drop
        drop_index = None

        # Find the index where the drop in frequency is greater than 1.0
        for i in range(1, len(frequency_list)):
            if frequency_list[i-1] - frequency_list[i] > 1.0:
                drop_index = i
                break

        # Take subset of word_list from the drop index onward
        subset_word_list = word_list[drop_index:] if drop_index is not None else []
        if len(subset_word_list) == 0:
            subset_word_list.append('')

        if subset_word_list[0] == 'pain':
            subset_word_list.insert(0, word_list[word_list.index('pain')-1])

        cc_final = ' '.join(subset_word_list)
        if len(cc_final) > 50:
            cc_final = cc_final[0:50]
        ## OUTPUT LABEL -- should contain the chief complaint
        cc_fisher.append(cc_final)

        freq_subsetter_idx+=1

    return cc_fisher

#cc_fisher = frequency_parser(lo_tkn, lo_freqs)
#cc_fisher[0:10]

##### Now let's compare the extracted chief complaints in the virtassist training set with the ground truth chief complaints.

In [ ]:
df_train_virtassist['chief_complaint'].values[0:10]

### Putting this altogether for chief complaint extraction...

In [ ]:
def chief_complaint_from_dialogue(df, word_list = ['present', 'eval', 'here for', 'here with', 'for']):

    start = time.time()

    noncomplacent_inds, kwarg_occurence_count, reading_frames = kwarg_counter('dialogue', word_list, df)
    lo_frames = reading_frame_cleaner(reading_frames)
    lo_tkn = frame_tokenizer(lo_frames)
    lo_freqs = freq_calc(lo_tkn)
    cc_fisher = frequency_parser(lo_tkn, lo_freqs)

    end = time.time()
    print("Time taken: " + str(end - start) + " seconds")

    return cc_fisher, [lo_tkn, lo_freqs]


##### Check that the code runs appropriately on training set

In [ ]:
temp_a, temp_b = chief_complaint_from_dialogue(df_train)
temp_a[0:10]

### Creating memory and time usage data

In [ ]:
import time
import tracemalloc

def chief_complaint_from_dialogue_stats(df, word_list=['present', 'eval', 'here for', 'here with', 'for']):
    start = time.time()
    tracemalloc.start()  # Start tracking memory

    # Run your analysis pipeline
    noncomplacent_inds, kwarg_occurence_count, reading_frames = kwarg_counter('dialogue', word_list, df)
    lo_frames = reading_frame_cleaner(reading_frames)
    lo_tkn = frame_tokenizer(lo_frames)
    lo_freqs = freq_calc(lo_tkn)
    cc_fisher = frequency_parser(lo_tkn, lo_freqs)

    current, peak = tracemalloc.get_traced_memory()  # Get memory in bytes
    tracemalloc.stop()

    end = time.time()
    total_time = end - start
    peak_memory_MB = peak / 1024 / 1024  # Convert bytes to megabytes

    return total_time, peak_memory_MB


In [ ]:
time_runs_srm=[]
memory_runs_srm=[]
for i in range(200):
  time_i, memory_i=chief_complaint_from_dialogue_stats(df_test.sample(frac=1, random_state=i),
                              word_list= ['present', 'eval', 'here for', 'here with', 'with', 'for'])
  time_runs_srm.append(time_i)
  memory_runs_srm.append(memory_i)

print(np.mean(time_runs_srm) * 1000,' ms')

std_dev = np.std(time_runs_srm, ddof=1)
n = np.size(time_runs_srm)
sem = std_dev / np.sqrt(n)

print(sem * 1000,' ms')
#print(np.mean(memory_runs_srm),' MB')

In [ ]:
mem_after = process.memory_info().rss

In [ ]:
memory_used_MB = (mem_after - mem_before) / 1024 / 1024
memory_used_MB

# LLMs
Instead of re-training or fine-tuning the model on our clinical documentation dataset, we evaluated its zero-shot capability to generate the chief complaints based on the dialouge. 

## Load necessary libraries and data

In [ ]:
# set groq API
api_key = "YOUR_GROQ_API_KEY"
client = groq.Client(api_key=api_key)

## Start experiment-specific loading files and function
This is when I start recording memory usage

Model run limits: https://console.groq.com/docs/rate-limits

to-do: This needs to be updated to run more than 1 iteration per run, need developer API Key

In [ ]:
def measure_LLM_performance(model_type, prompt, df_test):
  start = time.time()
  # get dialouge and chief complaint from test data
  inputs = np.array(df_test['dialogue'].tolist())
  labels = np.array(df_test['chief_complaint'].tolist())
  # holder for 11 category labels, run-time, and LLM prediction
  list_category_labels = []
  list_category_preds = []
  total_prompt_tokens = 0
  total_completion_tokens = 0

  # Loop through test set and use LLM to get the predictions
  for index in range(df_test.shape[0]):
      transcription = inputs[index]
      label = labels[index]
      category_label = labelMap.get(label.lower().rstrip(), 'others')

      # call groq api
      chat_completion = client.chat.completions.create(
          messages=[
              {"role": "system", "content": prompt},
              {"role": "user", "content": transcription}
          ],
          model=model_type,
      )

      # get the response from LLM
      pred = chat_completion.choices[0].message.content
      usage = chat_completion.usage  # Track token usage
      total_prompt_tokens += usage.prompt_tokens
      total_completion_tokens += usage.completion_tokens

      list_category_labels.append(category_label)
      list_category_preds.append(pred)

  end = time.time()
  total_time = end - start

  total_tokens = total_prompt_tokens + total_completion_tokens

  # https://groq.com/pricing
  if model_type == "llama-3.3-70b-versatile":
    cost = (total_prompt_tokens / 1e6 * 0.59) + (total_completion_tokens / 1e6 * 0.79)
  elif model_type == "llama3-70b-8192":
    cost = (total_prompt_tokens / 1e6 * 0.59) + (total_completion_tokens / 1e6 * 0.79)
  elif model_type == "gemma2-9b-it":
    cost = (total_prompt_tokens / 1e6 * 0.20) + (total_completion_tokens / 1e6 * 0.20)
  else:
    print("Models don't match, cost can't be calculated")

  return total_time, total_tokens, cost, total_prompt_tokens, total_completion_tokens


def run_model_prompt(model_type_list, prompt_list, prompt_names_list):

  process = psutil.Process(os.getpid())
  # Measure memory before loading the model
  mem_before = process.memory_info().rss

  #Load data
  train_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/clinical_training.csv', index_col=0)
  test_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/clinical_testing.csv', index_col=0)

  #Set up variables
  time_runs_list=[]
  memory_used_list=[]
  total_input_tokens_list=[]
  total_output_tokens_list=[]
  total_tokens_list=[]
  total_costs_list=[]

  for run in np.arange(len(model_type_list)):
    model_type = model_type_list[run]
    system_promp = prompt_list[run]
    prompt_name = prompt_names_list[run]

    print(f"Run {run+1}: Model {model_type} on {prompt_name}")

    time_runs=[]
    total_input_tokens=[]
    total_output_tokens=[]
    total_tokens=[]
    total_costs=[]
    for i in range(30):
      time_i, tokens_i, cost_i, input_token_i, output_token_i =measure_LLM_performance(model_type, system_promp, test_df.sample(frac=1, random_state=i))
      time_runs.append(time_i)
      total_tokens.append(tokens_i)
      total_costs.append(cost_i)
      total_input_tokens.append(input_token_i)
      total_output_tokens.append(output_token_i)


    # Save each run into these lists
    time_runs_list.append(time_runs)
    total_input_tokens_list.append(total_input_tokens)
    total_output_tokens_list.append(total_output_tokens)
    total_tokens_list.append(total_tokens)
    total_costs_list.append(total_costs)

    print(np.mean(time_runs) * 1000,' ms')


    std_dev = np.std(time_runs, ddof=1)
    n = np.size(time_runs)
    sem = std_dev / np.sqrt(n)

    print(sem * 1000,' ms')

    mem_after = process.memory_info().rss

    memory_used_MB = (mem_after - mem_before) / 1024 / 1024
    print(memory_used_MB, 'MB')
    memory_used_list.append(memory_used_MB)

    std_dev_cost = np.std(total_costs, ddof=1)
    n_cost = np.size(total_costs)
    sem_cost = std_dev_cost / np.sqrt(n_cost)

    print(np.mean(total_tokens), 'total tokens')
    print('$',np.mean(total_costs))
    print('$',sem_cost)
    print("\n\n\n")
    print('-----------------------------------')

  return time_runs_list, memory_used_list, total_input_tokens_list, total_output_tokens_list, total_tokens_list, total_costs_list

In [ ]:
# define model type and content for system
#model_type = "llama-3.3-70b-versatile"
#model_type = "llama3-70b-8192" # only for estimating when other model is overused
#model_type = "gemma2-9b-it"
# prompt A
system_promp_A =  """
You are a doctor. Your task is to identify the patient's chief complaint based on the following dialogue between a doctor and a patient.
Please select a direct quotation from the dialogue which best identifies the chief complaint. You may use ellipses "(...)" to skip irrelevant dialogue
within the detected chief complaint. Simply provide your best assessment. No explanation is needed. """
system_promp_A = system_promp_A.replace("\n", " ")

# Prompt B

system_promp_B =  """
You are a doctor. Your task is to identify the patient's chief complaint based on the following dialogue between a doctor and a patient.
Please select a direct quotation from the dialogue which best identifies the chief complaint. You may use ellipses "(...)" to skip irrelevant dialogue
within the detected chief complaint. Simply provide your best assessment. No explanation is needed. Be concise; detect 10 or fewer words to summarize the chief complaint. """
system_promp_B = system_promp_B.replace("\n", " ")

In [ ]:
model_type_list=["gemma2-9b-it","gemma2-9b-it","llama-3.3-70b-versatile","llama-3.3-70b-versatile"]
prompt_list=[system_promp_A,system_promp_B,system_promp_A,system_promp_B]
prompt_names_list=['Verbose', 'Succinct', 'Verbose', 'Succinct']

In [ ]:
#Set up variables
time_runs_list=[]
memory_used_list=[]
total_input_tokens_list=[]
total_output_tokens_list=[]
total_tokens_list=[]
total_costs_list=[]

time_runs_list, memory_used_list, total_input_tokens_list, total_output_tokens_list, total_tokens_list, total_costs_list = run_model_prompt(model_type_list, prompt_list, prompt_names_list)

## Plot the Data
Need to rework this to adjust for the new lists of outputs in run_model_prompt function

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ['Symbolic Model', 'Gemma-9b-succinct', 'Gemma-9b-Verbose', 'Llama-3.3-70b-succint', 'Llama-3.3-70b-Verbose']
run_time_ms = [4.091320037841797, 72480.13353347778,71777.21548080444,101445.04475593567,146168.6553955078]  # Time in milliseconds
peak_memory_MB = [21.3046875,0.47265625,0.50390625,0.46875,0.53515625]   # Memory usage in megabytes

x = np.arange(len(models))  # the label locations
width = 0.35  # width of the bars

fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for run time
bars1 = ax1.bar(x - width/2, run_time_ms, width, label='Run Time (ms)', color='#4878d0', alpha=0.8)

# Second Y-axis for memory
ax2 = ax1.twinx()
bars2 = ax2.bar(x + width/2, peak_memory_MB, width, label='Peak Memory (MB)', color='#d65f5f', alpha=0.8)

# Labels and titles
ax1.set_xlabel('Model')
ax1.set_ylabel('Run Time (ms)', color='#4878d0')
ax2.set_ylabel('Peak Memory (MB)', color='#d65f5f')
ax1.set_title('Run Time and Memory Usage per Model')
ax1.set_xticks(x)
ax1.set_xticklabels(models)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ['Symbolic Model', 'Gemma-9b-succinct', 'Gemma-9b-Verbose', 'Llama-3.3-70b-succint', 'Llama-3.3-70b-Verbose']
run_time_ms = [4.09, 72480.13, 71777.22, 101445.04, 146168.66]  # Time in milliseconds
peak_memory_MB = [21.30, 0.47, 0.50, 0.47, 0.54]   # Memory usage in megabytes

# Example standard deviation or error margins (replace with real values!)
run_time_std = [0.5, 1500, 1600, 2000, 2500]
peak_memory_std = [0.1, 0.02, 0.03, 0.02, 0.04]


x = np.arange(len(models))
width = 0.35

fig, ax1 = plt.subplots(figsize=(12, 7))

bars1 = ax1.bar(x - width/2, run_time_ms, width,
                yerr=run_time_std, capsize=5,
                label='Run Time (ms)', color='#4878d0', alpha=0.8)

ax2 = ax1.twinx()

bars2 = ax2.bar(x + width/2, peak_memory_MB, width,
                yerr=peak_memory_std, capsize=5,
                label='Peak Memory (MB)', color='#d65f5f', alpha=0.8)


# Log scale
ax1.set_yscale('log')

# Axis labels and ticks
ax1.set_xlabel('Model')
ax1.set_ylabel('Run Time (ms)')
ax2.set_ylabel('Peak Memory (MB)')
ax1.set_title('Run Time and Memory Usage per Model')
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=15, ha='right')

# Legend
bars = [bars1[0], bars2[0]]
labels = ['Run Time (ms)', 'Peak Memory (MB)']
ax1.legend(bars, labels, loc='upper left')

# Significance logic
def get_sig_star(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    return 'ns'

# # --- 1. Run Time p-values (on ax1) ---
# rt_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]
# rt_p_values = [0.01, 0.005, 0.0005, 0.0001]
# v_offset_rt = 4000  # vertical spacing between each line

# for idx, ((i, j), p) in enumerate(zip(rt_comparisons, rt_p_values)):
#     base_y = max(run_time_ms[i], run_time_ms[j]) + 5000
#     h = 3000 + idx * v_offset_rt
#     ax1.plot([x[i] - width/2, x[i] - width/2, x[j] - width/2, x[j] - width/2],
#              [base_y + h, base_y + h + 1000, base_y + h + 1000, base_y + h],
#              lw=1.5, color='black')
#     ax1.text((x[i] + x[j]) / 2 - width/2, base_y + h + 1100,
#              get_sig_star(p), ha='center', va='bottom', fontsize=12, color='black')

# --- 2. Peak Memory p-values (on ax2) ---
pm_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]
pm_p_values = [0.03, 0.02, 0.04, 0.045]
v_offset_pm = 0.8  # vertical spacing between each line

for idx, ((i, j), p) in enumerate(zip(pm_comparisons, pm_p_values)):
    base_y = max(peak_memory_MB[i], peak_memory_MB[j]) + 0.5
    h = 0.5 + idx * v_offset_pm
    ax2.plot([x[i] + width/2, x[i] + width/2, x[j] + width/2, x[j] + width/2],
             [base_y + h, base_y + h + 0.2, base_y + h + 0.2, base_y + h],
             lw=1.5, color='black')
    ax2.text((x[i] + x[j]) / 2 + width/2, base_y + h + 0.25,
             get_sig_star(p), ha='center', va='bottom', fontsize=12, color='black')



plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ['Symbolic Model', 'Gemma-9b-succinct', 'Gemma-9b-Verbose', 'Llama-3.3-70b-succint', 'Llama-3.3-70b-Verbose']
run_time_ms = [5.74, 65408.52, 63359.42, 5777.58, 6720.83]
peak_memory_MB = [21.30, 6.625, 6.109, 6.882, 6.882]

run_time_std = [0.1, 75.63, 1976.70, 48.26, 58.76]
peak_memory_std = [0.0, 0.0, 0.0, 0.0, 0.0]

x = np.arange(len(models))
width = 0.6

def get_sig_star(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    return 'ns'

def draw_sig(ax, xi, xj, y, h, label, dx=0.0, lw=1.5):
    """Draw a significance bracket with optional horizontal offset dx."""
    x1, x2 = xi + dx, xj + dx
    ax.plot([x1, x1, x2, x2],
            [y,  y + h, y + h, y],
            lw=lw, color='black', clip_on=False)
    ax.text((x1 + x2) / 2, y + h * 1.05, label,
            ha='center', va='bottom', fontsize=12, color='black', clip_on=False)

# --- Plot 1: Run Time (log) — multiplicative spacing ---
fig1, ax1 = plt.subplots(figsize=(10, 8))
bars1 = ax1.bar(x, run_time_ms, yerr=run_time_std, capsize=5, color='#4878d0', alpha=0.85)
ax1.set_yscale('log')
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=15, ha='right')
ax1.set_ylabel('Run Time (ms)')
ax1.set_title('Run Time per Model')
ax1.set_xlabel('Model')

rt_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]
rt_p_values    = [0.01, 0.005, 0.0005, 0.0001]
rt_dx = np.linspace(-0.18, 0.18, len(rt_comparisons))

# Key knobs for log spacing
above_frac   = 0.10   # start each bracket 10% above taller of the pair
level_factor = 1.7    # how much higher each successive bracket sits (uniform in log space)
h_frac       = 0.12   # bracket height as a fraction of its baseline level

y_needed_max = ax1.get_ylim()[1]
for idx, ((i, j), p) in enumerate(zip(rt_comparisons, rt_p_values)):
    pair_top = max(run_time_ms[i], run_time_ms[j])
    # multiplicative level in log space
    level = pair_top * (1.0 + above_frac) * (level_factor ** idx)
    h = level * h_frac
    draw_sig(ax1, x[i], x[j], level, h, get_sig_star(p), dx=rt_dx[idx])
    y_needed_max = max(y_needed_max, level + 1.25*h)

# ensure headroom on log axis
ymin, ymax = ax1.get_ylim()
if y_needed_max > ymax:
    ax1.set_ylim(ymin, y_needed_max)

plt.tight_layout()
plt.show()


# --- Plot 2: Peak Memory ---
fig2, ax2 = plt.subplots(figsize=(10, 8))
bars2 = ax2.bar(x, peak_memory_MB, capsize=5, color='#d65f5f', alpha=0.85)
ax2.set_xticks(x)
ax2.set_xticklabels(models, rotation=15, ha='right')
ax2.set_ylabel('Peak Memory (MB)')
ax2.set_title('Peak Memory Usage per Model')
ax2.set_xlabel('Model')

pm_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]
pm_p_values     = [0.03, 0.02, 0.04, 0.045]

# Jitters for memory brackets
pm_dx = np.linspace(-0.18, 0.18, len(pm_comparisons))

v_offset_pm = 0.5   # bracket height
gap_pm      = 0.5   # space above tallest of the pair
y_needed_max = 0

for idx, ((i, j), p) in enumerate(zip(pm_comparisons, pm_p_values)):
    base_y = max(peak_memory_MB[i], peak_memory_MB[j]) + gap_pm
    h = v_offset_pm + idx * 1.5
    draw_sig(ax2, x[i], x[j], base_y, h, get_sig_star(p), dx=pm_dx[idx])
    y_needed_max = max(y_needed_max, base_y + h * 1.2)

ymin, ymax = ax2.get_ylim()
if y_needed_max > ymax:
    ax2.set_ylim(ymin, y_needed_max)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind_from_stats  # NEW: to compute p-values from summary stats

models = ['Symbolic Model', 'Gemma-9b-succinct', 'Gemma-9b-Verbose', 'Llama-3.3-70b-succint', 'Llama-3.3-70b-Verbose']
run_time_ms = [5.74, 65408.52, 63359.42, 5777.58, 6720.83]
peak_memory_MB = [21.30, 6.625, 6.109, 6.882, 6.882]

run_time_std = [0.1, 75.63, 1976.70, 48.26, 58.76]
peak_memory_std = [0.0, 0.0, 0.0, 0.0, 0.0]

# --- Sample size per model (given) ---
n = 30

x = np.arange(len(models))
width = 0.6

def get_sig_star(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    return 'ns'

def draw_sig(ax, xi, xj, y, h, label, dx=0.0, lw=1.5):
    """Draw a significance bracket with optional horizontal offset dx."""
    x1, x2 = xi + dx, xj + dx
    ax.plot([x1, x1, x2, x2],
            [y,  y + h, y + h, y],
            lw=lw, color='black', clip_on=False)
    ax.text((x1 + x2) / 2, y + h * 1.05, label,
            ha='center', va='bottom', fontsize=12, color='black', clip_on=False)

def safe_std(s):
    # Avoid zero-variance crashing Welch's t-test
    return s if s > 0 else 1e-9

# --- Plot 1: Run Time (log) — multiplicative spacing ---
fig1, ax1 = plt.subplots(figsize=(10, 8))
bars1 = ax1.bar(x, run_time_ms, yerr=run_time_std, capsize=5, color='#4878d0', alpha=0.85)
ax1.set_yscale('log')
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=15, ha='right')
ax1.set_ylabel('Run Time (ms)')
ax1.set_title('Run Time per Model')
ax1.set_xlabel('Model')

rt_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]
rt_dx = np.linspace(-0.18, 0.18, len(rt_comparisons))

# Key knobs for log spacing
above_frac   = 0.10   # start each bracket 10% above taller of the pair
level_factor = 1.7    # how much higher each successive bracket sits (uniform in log space)
h_frac       = 0.12   # bracket height as a fraction of its baseline level

y_needed_max = ax1.get_ylim()[1]
for idx, (i, j) in enumerate(rt_comparisons):
    # Compute p-value from means/stds with n=30 using Welch's t-test
    _, p = ttest_ind_from_stats(mean1=run_time_ms[i], std1=safe_std(run_time_std[i]), nobs1=n,
                                mean2=run_time_ms[j], std2=safe_std(run_time_std[j]), nobs2=n,
                                equal_var=False)

    pair_top = max(run_time_ms[i], run_time_ms[j])
    level = pair_top * (1.0 + above_frac) * (level_factor ** idx)
    h = level * h_frac
    draw_sig(ax1, x[i], x[j], level, h, get_sig_star(p), dx=rt_dx[idx])
    y_needed_max = max(y_needed_max, level + 1.25*h)

# ensure headroom on log axis
ymin, ymax = ax1.get_ylim()
if y_needed_max > ymax:
    ax1.set_ylim(ymin, y_needed_max)

plt.tight_layout()
plt.show()

# --- Plot 2: Peak Memory ---
fig2, ax2 = plt.subplots(figsize=(10, 8))
bars2 = ax2.bar(x, peak_memory_MB, capsize=5, color='#d65f5f', alpha=0.85)
ax2.set_xticks(x)
ax2.set_xticklabels(models, rotation=15, ha='right')
ax2.set_ylabel('Peak Memory (MB)')
ax2.set_title('Peak Memory Usage per Model')
ax2.set_xlabel('Model')

pm_comparisons = [(0, 1), (0, 2), (0, 3), (0, 4)]

# Jitters for memory brackets
pm_dx = np.linspace(-0.18, 0.18, len(pm_comparisons))

v_offset_pm = 0.5   # bracket height
gap_pm      = 0.5   # space above tallest of the pair
y_needed_max = 0

for idx, (i, j) in enumerate(pm_comparisons):
    # Compute p-value from means/stds with n=30 using Welch's t-test
    _, p = ttest_ind_from_stats(mean1=peak_memory_MB[i], std1=safe_std(peak_memory_std[i]), nobs1=n,
                                mean2=peak_memory_MB[j], std2=safe_std(peak_memory_std[j]), nobs2=n,
                                equal_var=False)

    base_y = max(peak_memory_MB[i], peak_memory_MB[j]) + gap_pm
    h = v_offset_pm + idx * 1.5
    draw_sig(ax2, x[i], x[j], base_y, h, get_sig_star(p), dx=pm_dx[idx])
    y_needed_max = max(y_needed_max, base_y + h * 1.2)

ymin, ymax = ax2.get_ylim()
if y_needed_max > ymax:
    ax2.set_ylim(ymin, y_needed_max)

plt.tight_layout()
plt.show()
